In [1]:
import re
import tensorflow as tf
import numpy as np
import time
import os
import collections
import random
import time
import nltk
import pickle

In [2]:
def clearstring(string):
    string = re.sub('[^A-Za-z\- ]+', '', string)
    string = string.split(' ')
    string = filter(None, string)
    string = [y.strip() for y in string]
    return [y.lower() for y in string if len(y) > 3 and y.find('nbsp') < 0]

def build_dataset(words, n_words):
    count = [['UNK', -1]]
    count.extend(collections.Counter(words).most_common(n_words - 1))
    dictionary = dict()
    for word, _ in count:
        dictionary[word] = len(dictionary)
    data = list()
    unk_count = 0
    for word in words:
        index = dictionary.get(word, 0)
        if index == 0:  # dictionary['UNK']
            unk_count += 1
        data.append(index)
    count[0][1] = unk_count
    reversed_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
    return data, count, dictionary, reversed_dictionary

data_index = 0

# Step 3: Function to generate a training batch for the skip-gram model.
def generate_batch(batch_size, num_skips, skip_window):
    global data_index
    global data
    assert batch_size % num_skips == 0
    assert num_skips <= 2 * skip_window
    batch = np.ndarray(shape=(batch_size), dtype=np.int32)
    labels = np.ndarray(shape=(batch_size, 1), dtype=np.int32)
    span = 2 * skip_window + 1  # [ skip_window target skip_window ]
    buffer = collections.deque(maxlen=span)
    if data_index + span > len(data):
        data_index = 0
    buffer.extend(data[data_index:data_index + span])
    data_index += span
    for i in range(batch_size // num_skips):
        context_words = [w for w in range(span) if w != skip_window]
        words_to_use = random.sample(context_words, num_skips)
        for j, context_word in enumerate(words_to_use):
            batch[i * num_skips + j] = buffer[skip_window]
            labels[i * num_skips + j, 0] = buffer[context_word]
        if data_index == len(data):
            for word in data[:span]:
                buffer.append(word)
            data_index = span
        else:
            buffer.append(data[data_index])
            data_index += 1
    # Backtrack a little bit to avoid skipping words in the end of a batch
    data_index = (data_index + len(data) - span) % len(data)
    return batch, labels

In [3]:
with open('books/Catching-Fire.txt', 'r',encoding = "ISO-8859-1") as fopen:
    vocabulary = clearstring(' '.join(nltk.word_tokenize(fopen.read())))
print('example 10 words:',vocabulary[:10])
print('size corpus:',len(vocabulary))
vocabulary_size = len(list(set(vocabulary)))
print('size of unique words:',vocabulary_size)
dimension = 128
skip_window = 1
num_skips = 2
batch_size = 64
location = os.getcwd()

example 10 words: ['catching', 'fire', 'part', 'spark', 'clasp', 'flask', 'between', 'hands', 'even', 'though']
size corpus: 55085
size of unique words: 7543


In [4]:
data, count, dictionary, reverse_dictionary = build_dataset(vocabulary,vocabulary_size)
del vocabulary  # Hint to reduce memory.
print('Most common words (+UNK)', count[:5])
print('Sample data', data[:10], [reverse_dictionary[i] for i in data[:10]])

Most common words (+UNK) [['UNK', 1], ('that', 1008), ('with', 758), ('peeta', 664), ('have', 551)]
Sample data [1440, 215, 294, 1595, 4796, 2077, 177, 94, 24, 118] ['catching', 'fire', 'part', 'spark', 'clasp', 'flask', 'between', 'hands', 'even', 'though']


In [5]:
valid_size = 16
valid_window = 100
valid_examples = np.random.choice(valid_window, valid_size, replace=False)

In [6]:
graph = tf.Graph()

with graph.as_default():

    train_inputs = tf.placeholder(tf.int32, shape=[batch_size])
    train_labels = tf.placeholder(tf.int32, shape=[batch_size, 1])
    valid_dataset = tf.constant(valid_examples, dtype=tf.int32)

    # Ops and variables pinned to the CPU because of missing GPU implementation
    with tf.device('/cpu:0'):
        embeddings = tf.Variable(tf.random_uniform([vocabulary_size, dimension], -1.0, 1.0))
        embed = tf.nn.embedding_lookup(embeddings, train_inputs)
        nce_weights = tf.Variable(tf.truncated_normal([vocabulary_size, dimension],
                                                      stddev=1.0 / np.sqrt(dimension)))
        nce_biases = tf.Variable(tf.zeros([vocabulary_size]))
        loss = tf.reduce_mean(tf.nn.nce_loss(weights=nce_weights,
                                             biases=nce_biases,
                                             labels=train_labels,
                                             inputs=embed,
                                             num_sampled=batch_size / 2,
                                             num_classes=vocabulary_size))
        optimizer = tf.train.GradientDescentOptimizer(1.0).minimize(loss)

        # Compute the cosine similarity between minibatch examples and all embeddings.
        norm = tf.sqrt(tf.reduce_sum(tf.square(embeddings), 1, keep_dims=True))
        normalized_embeddings = embeddings / norm
        valid_embeddings = tf.nn.embedding_lookup(normalized_embeddings, valid_dataset)
        similarity = tf.matmul(valid_embeddings, normalized_embeddings, transpose_b=True)
        init = tf.global_variables_initializer()

num_steps = 200000

In [7]:
with tf.Session(graph=graph) as session:
    init.run()
    print('Initialized')

    average_loss = 0
    for step in range(num_steps):
        batch_inputs, batch_labels = generate_batch(batch_size, num_skips, skip_window)
        feed_dict = {train_inputs: batch_inputs, train_labels: batch_labels}

        _, loss_val = session.run([optimizer, loss], feed_dict=feed_dict)
        average_loss += loss_val

        if step % 2000 == 0:
            if step > 0:
                average_loss /= 2000
            print('Average loss at step ', step, ': ', average_loss)
            average_loss = 0

        if step % 10000 == 0:
            sim = similarity.eval()
            for i in range(valid_size):
                valid_word = reverse_dictionary[valid_examples[i]]
                top_k = 8  # number of nearest neighbors
                nearest = (-sim[i, :]).argsort()[1:top_k + 1]
                log_str = 'Nearest to %s:' % valid_word
                for k in range(top_k):
                    close_word = reverse_dictionary[nearest[k]]
                    log_str = '%s %s,' % (log_str, close_word)
                print(log_str)
    final_embeddings = normalized_embeddings.eval()

Initialized
Average loss at step  0 :  98.9104537964
Nearest to president: heavily, scared, eighteen, reverse, creatures, pouring, there, heavier,
Nearest to back: concealed, bake, cheeses, referencing, resulting, scoured, oblivious, snapping,
Nearest to beetee: scowl, fourteen, streaks, greeting, terribly, expressions, onerous, shaky,
Nearest to even: embedded, favorite, unless, scale, daylong, revolting, shoos, forgot,
Nearest to because: inch, dust, tricks, broth, emptiness, good-bye, forty-eight, foolishness,
Nearest to come: muddy, falling, headed, maneuvering, badly, glare, kept, reply,
Nearest to cinna: brightest, granary, cane, feathers, unresponsive, carnivorous, reduced, third,
Nearest to take: entire, bundle, insubstantial, betting, demanding, this, fell, fairly,
Nearest to then: took, savor, evading, five-foot-high, snarls, experiencing, uncomprehendingly, cards,
Nearest to johanna: bags, massaged, self-centered, admits, half-emptied, bidding, inch, luring,
Nearest to mothe

Average loss at step  44000 :  2.55901190984
Average loss at step  46000 :  2.54310526019
Average loss at step  48000 :  2.50334929299
Average loss at step  50000 :  2.48377150542
Nearest to president: frothy, scared, fervently, heavily, popping, condemn, pouring, reverse,
Nearest to back: repay, snapping, attendant, olive, flung, road, blisters, techniques,
Nearest to beetee: provides, doing, slip, bone, scowl, committing, outlast, shaky,
Nearest to even: result, begs, valuable, shoos, dipping, saved, brrr, popular,
Nearest to because: studying, foolishness, seat, tricks, flee, good-bye, bedroom, mediocre,
Nearest to come: waterproof, goods, barbarism, muddy, blister, badly, funny, pantry,
Nearest to cinna: cane, examines, foolish, baskets, guaranteed, happy, third, clean,
Nearest to take: resist, jump, shoves, gossipy, elaborately, bundle, butter, served,
Nearest to then: five-foot-high, submerges, cards, vegetables, savor, overalls, stylists, bouncing,
Nearest to johanna: embraces, 

Average loss at step  92000 :  2.1996512903
Average loss at step  94000 :  2.20825835931
Average loss at step  96000 :  2.19497986221
Average loss at step  98000 :  2.20208203107
Average loss at step  100000 :  2.18188257363
Nearest to president: frothy, fervently, scared, angers, heavier, mourn, condemn, merely,
Nearest to back: attendant, repay, snapping, flung, techniques, blisters, steal, ensemble,
Nearest to beetee: provides, bone, outlast, doing, slip, excel, thom, youre,
Nearest to even: brrr, shoos, result, valuable, begs, swarm, popular, treasonous,
Nearest to because: foolishness, studying, bedroom, mediocre, bizarre, legal, steel, senseless,
Nearest to come: goods, waterproof, pantry, prior, sausage, startled, blister, streets,
Nearest to cinna: cane, clean, foolish, guaranteed, examines, scissors, package, mother,
Nearest to take: resist, jump, gossipy, celebrate, droning, betting, bundle, butter,
Nearest to then: five-foot-high, submerges, stylists, hungriest, vibrate, bot

Average loss at step  142000 :  2.12306393448
Average loss at step  144000 :  2.12400869995
Average loss at step  146000 :  2.12674189311
Average loss at step  148000 :  2.13405224115
Average loss at step  150000 :  2.11997469842
Nearest to president: frothy, angers, fervently, heavier, condemn, humming, shiver, merely,
Nearest to back: attendant, repay, snapping, techniques, ensemble, squarely, blisters, flung,
Nearest to beetee: provides, thom, outlast, excel, slip, rebelling, soups, fourteen,
Nearest to even: brrr, shoos, swarm, begs, popular, valuable, configuration, result,
Nearest to because: foolishness, legal, studying, mediocre, polite, arrested, dreadful, erupt,
Nearest to come: goods, waterproof, startled, streets, fifteen-step, prior, instructed, barbarism,
Nearest to cinna: cane, package, clean, guaranteed, foolish, examines, madge, stealthily,
Nearest to take: resist, jump, droning, celebrate, desirable, betting, averting, gripped,
Nearest to then: five-foot-high, submerg

Average loss at step  192000 :  2.08317133704
Average loss at step  194000 :  2.1047415975
Average loss at step  196000 :  2.0819549284
Average loss at step  198000 :  2.1082173754


In [8]:
with open('catchingfire-list.p', 'wb') as fopen:
    pickle.dump(list(reverse_dictionary.values()), fopen)

In [9]:
with open('catchingfire-vector.p', 'wb') as fopen:
    pickle.dump(final_embeddings, fopen)